# CS5228 Assignment 2 - DBSCAN, AGNES & Association Rule Mining

Hello everyone, this assignment notebook covers DBSCAN, AGNES, Association Rule Mining (ARM). There are some code-completion tasks and question-answering tasks in this answer sheet. For code completion tasks, please write down your answer (i.e., your lines of code) between sentences that "Your code starts here" and "Your code ends here". The space between these two lines does not reflect the required or expected lines of code. For answers in plain text, you can refer to [this Markdown guide](https://medium.com/analytics-vidhya/the-ultimate-markdown-guide-for-jupyter-notebook-d5e5abf728fd) to customize the layout (although it shouldn't be needed).

When you work on this notebook, you can insert additional code cells (e.g., for testing) or markdown cells (e.g., to keep track of your thoughts). However, before the submission, please remove all those additional cells again. Thanks!

**Important:** 
* Rename and save this Jupyter notebook as **cs5228_a2_YourName_YourNUSNETID.ipynb** (e.g., **cs5228_a2_BobSmith_e12345678.ipynb**) before submission!
* Rename and save the script file *cs5228_a2_script.py* as **cs5228_a2_YourName_YourNUSNETID.py** (e.g., **cs5228_a2_BobSmith_e12345678.py**) before submission!
* Submission deadline is Sep 28, 11.59 pm. Late submissions will be penalized by 10% for each additional day. Failure to appropriately rename both files will yield a penalty of 1 Point. There is no need to use your full name if its a rather long; it's just  important to easily identify you in Canvas etc.

Please also add your NUSNET and student id in the code cell below. This is just to make any identification of your notebook doubly sure.

In [ ]:
student_id = ''
nusnet_id = ''

Here is an overview over the tasks to be solved and the points associated with each task. The notebook can appear very long and verbose, but note that a lot of parts provide additional explanations, documentation, or some discussion. The code and markdown cells you are supposed to complete are well, but you can use the overview below to double-check that you covered everything.

* **1 DBSCAN (15 Points)**
    * 1.1Implementing DBSCAN for Noise Detection (10 Points)
        * 1.1 a) Compute Core Points (5 Points)
        * 1.2 b) Compute Noise Points (5 Points)
    * 1.2 Questions about DBSCAN (5 Points)
        * 1.2 a) Basic Data Understanding with DBSCAN (3 Points)
        * 1.2 b) Finding all Cluster (2 Points)
* **2 AGNES (10 Points)**
    * 2.1 Interpreting Dendrograms (6 Points)
    * 2.2 Questions about AGNES (4 Points)
        * 2.2 a) Picking the Right Linkage Method (2 Points)
        * 2.2 b) Outlier Detection with AGNES(2 Points)
* **3 Association Rule Mining (25 Points)**
    * 3.1 Brute-Forcing Association Rule Mining (5 Points)
    * 3.2 Implementing Apriori Algorithm for Finding Frequent Itemsets (10 Points)
        * 3.2 a) Generating Candidate Itemsets $L_{k}$ (5 Points)
        * 3.2 b) Generating Frequent Itemsets $F_{1}$, $F_{2}$, $F_{3}$, ... (5 Points)
    * 3.3 ARM over Real-World Datasets -- Questions (10 Points)
        * 3.3 a) (3 Points)
        * 3.3 b) (3 Points)
        * 3.3 c) (2 Points)
        * 3.3 d) (2 Points)

## Setting up the Notebook

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

from efficient_apriori import apriori

from src.utils import *

**Important:** This notebook also requires you to complete in a separate `.py` script file. This keeps this notebook cleaner and simplifies testing your implementations for us. As you need to rename the file `cs5228_a2.py`, you also need to edit the import statement below accordingly.

In [ ]:
from cs5228_a2 import *
#from cs5228_a2_BobSmith_e12345678 import *  # <-- you will need to rename this accordingly

---

## 1 DBSCAN

The Density-Based Spatial Clustering of Applications with Noise (DBSCAN) algorithm is a clustering algorithm used in data mining and machine learning. It is designed to identify clusters in data that may have arbitrary shapes and handle outliers effectively. Here's a more detailed explanation of how the DBSCAN algorithm works:

* **Density-Based Clustering:** DBSCAN identifies clusters based on the density of data points in the feature space rather than assuming a specific cluster shape.

* **Core Points:** The algorithm starts by selecting a random data point. It then checks how many data points are within a specified distance (epsilon or ε) from this point. If the number of data points within this distance exceeds a predefined threshold called MinPts (including the data point itself), the selected point is classified as a "core point."

* **Border Points:** Data points that are within the epsilon distance of a core point but do not meet the MinPts criterion are classified as "border points." Border points can be part of a cluster but are located on the edges of the cluster.

* **Noise Points:** Data points that are neither core points nor border points are classified as "noise points" or outliers. These are data points that do not belong to any cluster.

* **Cluster Expansion:** The algorithm iteratively expands clusters by connecting core points and their density-reachable neighbors. Two core points are considered "density-reachable" if there is a path of core points connecting them within the epsilon distance. This process continues until no more core points can be added to a cluster.

* **Result:** The output of DBSCAN is a set of clusters, each consisting of core points and potentially including border points. Noise points are not assigned to any cluster.

DBSCAN is a valuable algorithm for clustering spatial data when the characteristics of the clusters are not well-known in advance and is commonly used in various applications, including geospatial analysis, image processing, and anomaly detection. In this section, we look into DBSCAN to implement a simplified variant instead of the "full" algorithm. For this, let's first load some random data for visualization and testing.

In [ ]:
X_dbscan_toy = pd.read_csv('data/a2-dbscan-toy-dataset.txt', header=None, sep=' ').to_numpy()

print('The shape of X_dbscan_toy is {}'.format(X_dbscan_toy.shape))

Now we can run scikit-learn's implementation of [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) on this dataset. Here we use `eps=0.1` and `min_samples=10` as values for the two main input parameters for DBSCAN that specify the minimum "density" of clusters.

In [ ]:
dbscan_clustering = DBSCAN(eps=0.1, min_samples=11).fit(X_dbscan_toy)

The points that are noise points are labeled with `-1`, while all points belonging to clusters are labeled with `0`, `1`, `2`, etc. So we can easily find the indices of all the points labeled as noise as follows:

In [ ]:
cluster_point_indices = np.argwhere(dbscan_clustering.labels_ >= 0).squeeze()
noise_point_indices = np.argwhere(dbscan_clustering.labels_ < 0).squeeze()

print('The indices of the points labeled as noise are: {}'.format(noise_point_indices))

Of course, we can also plot the results. Note that the figure below only highlights the points labeled as noise as red triangles; all points belonging to *some* clusters are in grey points (note that we do not care to which exact cluster these points belong).

In [ ]:
plt.figure()
plt.scatter(X_dbscan_toy[cluster_point_indices,0], X_dbscan_toy[cluster_point_indices,1], c='grey')
plt.scatter(X_dbscan_toy[noise_point_indices,0], X_dbscan_toy[noise_point_indices,1], c='red', marker='^', s=75)
plt.show()

Summing up, the red dots in the plots we define as noise or outliers as they are very dissimilar to the other data points. In practice, we would likely remove those noise points, treat them separately, or maybe perform additional preprocessing steps to potentially "denoise" the dataset. However, the steps of choice generally depend heavily on the exact data mining task. Here, we focus on the identification of noise points.

### 1.1 Implementing DBSCAN for Noise Detection (10 Points)

In the lecture, we covered the original algorithm of DBSCAN, which you can also find on [Wikipedia](https://en.wikipedia.org/wiki/DBSCAN). While not difficult to implement, it takes quite a couple of lines of codes to do so. For this assignment, however, we are only interested in the points of a dataset that DBSCAN considers noise (as illustrated above; the red dots in the previous plot). This includes that we do not have to care about

* how many clusters there are (the plot above hints at 3 clusters but it does not matter) *and*
* which non-noise points (the grey dots in the plot above) belong to which cluster

**Your task is to implement a modified/simplified version of DBSCAN to find all noise points in a dataset!** The skeleton of method `get_noise_dbscan()` you need to complete is found in the file `cs5228_a2.py` (before the appropriate renaming). The method takes data matrix `X` as well as the two basic parameters `eps` and `min_samples` as input parameters; we use the same naming as scikit-learn's implementation of [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html).  The output should be 2 lists of indices: (a) one containing the indices of all *core points* and (b) on containing the indices of all *noise points* in input dataset X.

**Important:**
* We only split this task into 2.1 a) and 2.1 b) to have intermediate results you can check for correctness (and potentially to better allow for partial marking). Our reference solutions first finds all core points and uses this information to find all noise points; hence the 2 separate code blocks for you to complete.
* However, if you have a better/faster/shorter/cooler/etc. solution, you are more than welcome to implement it and ignore the intermediate result of finding all core points. Only the result from 2.1 b) is important. This also means that you can ignore 2.1 a) and still get full marks if you correctly identify all noise points.
* If you have an alternative solution, please make sure that the method still returns the 2 output parameters `(core_point_indices, noise_point_indices)`. If you do not need to explicitly identify the core points, you can simply return `None` for `core_point_indices`.
* You can import any method `numpy`, `scipy`, `sklearn`, or `pandas` has to offer -- except for any ready-made implementation of DBSCAN, of course :). Please add any imports to the code cell at the top with the other imports. Hint: We already imported [`sklearn.metrics.pairwise.euclidean_distances`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html) for you.

We will benchmark your implementation as part of our Little Competitions to see whose solution is the fastest.

#### 1.1 a) Compute Core Points (5 Points)

As mentioned above, our reference solution first computes all core points. If you follow this approach, complete the respective part in the code of method `get_noise_dbscan()`. Some hints:
* Recall that we do not care to which cluster a core point a data sample belongs, only that is a core point in *some* cluster
* Have a look at method [`sklearn.metrics.pairwise.euclidean_distances`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html); it might make your life easier.

**Important:** Avoid using loops in the parts of the code you have to complete. If you use loops but the results are correct, there will be some minor deduction of points. Once you know what you need to do, it is almost guaranteed that `numpy`, `scipy`, or `sklearn` will provide a useful method.

In [ ]:
my_core_point_indices, _ = get_noise_dbscan(X_dbscan_toy, eps=0.1, min_samples=10)

print('Total number of core points: {}\n'.format(len(my_core_point_indices)))
print('The first 25 indices of the points labeled as core points:\n{}'.format(sorted(my_core_point_indices)[:25]))

The output of previous code cell should look like:
    
```
Total number of core points: 50

The first 25 indices of the points labeled as core points:
[1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24]
```

Note that `0`, `4`, and `27` are missing from this list since [`sklearn.cluster.DBSCAN`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) told us that these points are noise. Of course, also the border points are missing here, but [`sklearn.cluster.DBSCAN`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) does not return those explicitly.

#### 1.1 b) Compute Noise Points (5 Points)

Knowing the core points is useful but only an intermediate step. Now it is time to complete the method `get_noise_dbscan()` to compute the indices of all noise points in `X`. Again, our reference solution uses `core_point_indices` to accomplish this. If your implementation does not require the information about core points but returns the correct `noise_point_indices` then this is perfectly fine!

**Important:** Avoid using loops in the parts of the code you have to complete. If you use loops but the results are correct, there will be some minor deduction of points. Once you know what you need to do, it is almost guaranteed that `numpy`, `scipy`, or `sklearn` will provide a useful method.

In [ ]:
_, my_noise_point_indices = get_noise_dbscan(X_dbscan_toy, eps=0.1, min_samples=10)

print('Total number of noise points: {}\n'.format(len(my_noise_point_indices)))
print('The indices of all points labeled as noise points:\n{}'.format(sorted(my_noise_point_indices)))

The output of previous code cell should look like:

```
Total number of noise points: 10

The indices of all points labeled as noise points:
[0, 4, 27, 31, 33, 39, 43, 46, 51, 65]
```

Since we used the same values for `eps` and `min_samples`, this result matches the output we saw earlier when we used scikit-learn's implementation of [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) over the toy dataset.

### 1.2 Questions about DBSCAN

**1.2 a) Basic Data Understanding with DBSCAN (3 Points)**

Assume you have a dataset `X`, run DBSCAN, and get a clustering that contains a set of clusters and some noise points (there's no need to be more precise; it's only important that you don't get just noise). Let's also assume you create a new dataset `X_new` simply by shuffling `X`; no other changes. Now you run DBSCAN with the same parameters as before over `X_new` and get a different clustering, i.e., most of the clusters are not exactly the same as before.

**Describe what this information tells about the dataset and clustering!** This may include a brief discussion how changing the parameters of DBSCAN will likely affect the results.

**Your Answer:**

**1.2 b) Finding all Cluster (2 Points)**

Your method `get_noise_dbscan()` finds all the noise points in a dataset according to the definition of DBSCAN. Now let's assume you now want to find all clusters, i.e., the number of clusters and which noise point belongs to which cluster.

**2.2a) Describe how the result of `get_noise_dbscan()` may help you to find all clusters! (2 Points)** There is no need to implement anything; just a brief description how clusters can be found by already knowing all noise points.

**Your Answer:**

---

## 2 AGNES

AGNES, which stands for Agglomerative Nesting, is an agglomerative hierarchical clustering algorithm used in data analysis and machine learning. It is a bottom-up approach to clustering, meaning it starts with individual data points as separate clusters and iteratively merges them into larger clusters. Here's an overview of how the AGNES clustering algorithm works:

* **Initialization:** Initially, each data point is treated as a single cluster, resulting in as many clusters as there are data points.

* **Agglomerative Process:** AGNES proceeds by iteratively merging the two closest clusters into a single larger cluster. The distance between clusters is typically determined by a linkage criterion, which can be one of the following commonly used methods: *Single Linkage*, *Complete Linkage*, *Average Linkage*, and more.

* **Hierarchy Building:** As clusters are merged, a hierarchy or dendrogram is built to represent the sequence of cluster mergers. This dendrogram can be used to visualize the hierarchical structure of the data and choose the desired number of clusters later.

* **Stopping Criterion:** AGNES continues merging clusters until a stopping criterion is met. This criterion could be based on a predetermined number of clusters, a threshold distance, or other criteria. By default, AGNES stops when all data points form a single cluster.

* **Result:** The output of AGNES is a hierarchical tree-like structure called a dendrogram, which can be cut at a specific level to obtain a partition of the data into clusters. The choice of where to cut the dendrogram depends on the desired number of clusters.

AGNES is a valuable tool for hierarchical clustering and can be useful in various data analysis tasks, including taxonomy construction, gene expression analysis, and image segmentation.

### 2.1 Interpreting Dendrograms

We saw in the lecture that dendrograms are a meaningful way to visualize the hierarchical relationships between the data points with respect to the clustering using AGNES (or any other hierarchical clustering technique). Properly interpreting is important to get a correct understanding of the underlying data.

Below are the plots of 6 different datasets labeled A-F. Each dataset contains 30 data points, each with two dimensions.

<img src="data/a2-agnes-data-labeled.png">

Below are 6 dendrograms labeled 1-6. These dendrograms show the clustering using **AGNES with Single Linkage** for the 6 dataset above, but in a random order.

<img src="data/a2-agnes-dendrogram-labeled.png">

**Find the correct combinations of datasets and dendrograms** -- that is, find for each dataset the dendrogram that visualizes the clustering using AGNES with Single Linkage! Give a brief explanation for each decision; max 1-2 sentences! Complete the table below!

| Dataset | Dendrogram | Brief Explanation |
| ---  | ---   | ---                  |
| **A**    | ??? | ??? |
| **B**    | ??? | ??? |
| **C**    | ??? | ??? |
| **D**    | ??? | ??? |
| **E**    | ??? | ??? |
| **F**    | ??? | ??? |

### 2.2 Questions about AGNES (4 Points)

**2.2 a) Picking the Right Linkage Method (2 Points)**

Assume your dataset contains the geolocations of traffic accidents on Singapore expressways over the time span of a year. Using AGNES, we want to find sections of the expressways where traffic accidents are particularly common.

**Which Linkage Methods covered in the lecture is most suitable for this task?** Briefly explain your choice!


**Your Answer:**

**2.2 b) Outlier Detection with AGNES (2 Points)**

In contrast to DBSCAN, AGNES has no explicit concept of noise points for outlier detection. But an AGNES clustering still gives meaningful insights into a dataset.

**How can the results of AGNES be used to identify outliers in a data?** Briefly explain your answer!

**Your Answer:**

---

## 3 Association Rule Mining

Association rule mining is a data mining technique used to discover interesting relationships, patterns, or associations in large datasets. It primarily focuses on identifying frequent patterns, often in the form of rules, within transactional or relational databases. This technique is commonly applied in various fields, including market basket analysis in retail, web usage mining, healthcare, and more. Here's how association rule mining works:

* **Transaction Data:** Association rule mining typically starts with a dataset that represents transactions or events. Each transaction consists of a set of items, and these transactions are often stored in a database.

* **Itemsets:** An itemset is a collection of one or more items or attributes from the dataset. For example, in a retail context, an itemset could be a list of products purchased together in a single transaction.

* **Support:** Support is a measure of the frequency with which an itemset appears in the dataset. It is defined as the proportion of transactions that contain the itemset. High support indicates that the itemset is a common pattern in the data.

* **Confidence:** Confidence is a measure of how often the rule is found to be true. It is defined as the proportion of transactions that contain both the antecedent and the consequent. High confidence suggests a strong association between the antecedent and consequent.

* **Lift:** Lift is a measure of how much more likely the consequent is given the antecedent compared to its expected occurrence by chance. A lift value greater than 1 suggests a positive association, while a value less than 1 suggests a negative or weak association.

* **Association Rules:** Association rules are typically expressed as "if-then" statements. They describe relationships between itemsets. Each rule consists of two parts:
    * Antecedent (Left-hand side, LHS): This is the itemset that represents the condition or premise of the rule.
    * Consequent (Right-hand side, RHS): This is the itemset that represents the result or conclusion of the rule.

The association rule mining process involves finding itemsets that meet a minimum support threshold and then generating rules based on those itemsets. Rules are evaluated based on their confidence and lift, and interesting or meaningful rules are selected for further analysis or action. For example, in a retail context, association rule mining might reveal that customers who purchase items like bread and milk (antecedent) are highly likely to also purchase butter (consequent). This insight can be valuable for marketing and inventory management, as it suggests items that can be promoted together or placed in close proximity in the store. Association rule mining algorithms, such as the Apriori algorithm, are commonly used to efficiently discover and analyze these patterns in large datasets.

#### Toy Dataset

The following dataset with 5 transactions and 6 different items is directly taken from the lecture slides. This should make it easier to test your implementation. The format is a list of tuples, where each tuple represents the set of items of an individual transaction. This format can also be used as input for the `efficient-apriori` package.

In [ ]:
transactions_demo = [
    ('bread', 'yogurt'),
    ('bread', 'milk', 'cereal', 'eggs'),
    ('yogurt', 'milk', 'cereal', 'cheese'),
    ('bread', 'yogurt', 'milk', 'cereal'),
    ('bread', 'yogurt', 'milk', 'cheese')
]

#### Auxiliary Methods

We want you to focus on the Apriori algorithm. So we provide you with a set of auxiliary functions. Feel free to look at their implementation in the file `data/utils.py`.

Given a set of items, `powerset()` returns all possible subsets of items with a specified minimum and maximum length. For example, you can use this method to generate all itemsets for a transaction.

In [ ]:
for subset in powerset(('c', 'b', 'a'), min_len=1, max_len=3):
    print(subset)

The method `unique_items()` returns all the unique items across all transactions.

In [ ]:
unique_items(transactions_demo)

The method `support()` calculates and returns the support for a given itemset and set of transactions.

In [ ]:
support(transactions_demo, ('bread', 'milk'))

The method `confidence()` calculates and returns the confidence for a given association rules and set of transactions. An association rule is represented by a 2-tuple, where the first element represents itemset X and the second element represents items Y (i.e., $X \Rightarrow Y$)

In [ ]:
confidence(transactions_demo, (('bread',), ('milk',)))

The method `generate_association_rules()` calculates and returns all possible association rules given an itemset. The result is a list of association rules, each association rule represented as 2-tuple (see above).

In [ ]:
generate_association_rules(('bread', 'milk', 'cereal'))

The method `merge_itemsets()` merges two given itemsets into one itemset.

In [ ]:
merge_itemsets(('bread', 'milk'), ('bread', 'eggs'))

For your implementation, you can make use of these auxiliary methods wherever you see fit. And that is, of course, strongly recommended, as it makes the programming task much easier. So, let's get started.

### 3.1 Brute-Forcing Association Rule Mining (5 Points)

The most naive approach for mining association rules would be to generate all possible rules and check if their support and confidence exceed the specified thresholds `minsup` and `minconf`. In the lecture, you have learned that, given $d$ unique items in a dataset of transactions, there are $3^d - 2^{d+1} + 1$ possible rules.

**Proof that $d$ unique items result in $3^d - 2^{d+1} + 1$ possible rules!** (Hint: Write out all possible rules for $d = 2, 3, 4, ...$ items; you should quickly spot the pattern that will allow you to validate the formula).

**Your Answer:**

### 3.2 Implementing Apriori Algorithm for Finding Frequent Itemsets (10 Points)

The underlying goal of the Apriori algorithm is to avoid calculating the support of itemsets for which we can say ahead of time (i.e., "a-priori") that they cannot be frequent itemsets. Calculating the support is the most expensive part as it requires going through all transactions in our database.

In the following task, you will implement the Apriori algorithm for finding Frequent Itemsets. In more detail, you will implement the $F_{k-1}\times F_1$ method to compute the $L_k$ candidate itemsets given the $F_{k-1}$ frequent itemsets and the $F_1$ frequent itemsets.

#### 3.2 a) Generating Candidate Itemsets $L_{k}$ (5 Points)

The $F_{k-1}\times F_1$ method used the frequent itemset sets of size ($k-1$) and of size $1$ to derive the set $L_k$ of candidate itemsets. The important part is the pruning, e.g., the removal of all k-itemset for which we can already tell that they won't have sufficient support.

**Hint:** Instead of first creating all possible k-itemsets and then performing the pruning to yield the final set $L_k$, check each k-itemset immediately in the loop if it should indeed be added to $L_k$ or not. This will make your code a bit simpler.

In [ ]:
def generate_Lk(k_minus_one_itemsets, one_itemsets):
    
    # Initialize as set to avoid duplicates
    Lk = set()
    
    # Nested loop over both itemsets to find all combinations
    for k_minus_one_itemset in k_minus_one_itemsets:
        for one_itemset in one_itemsets:
            
            #########################################################################################
            ### Your code starts here ###############################################################              
            

                
            ### Your code ends here #################################################################
            #########################################################################################                  
                
    return Lk

For testing your implementation, we manually crafted a k-itemset ($k=2$) and 1-itemset in line with the lecture example, meaning that the result should match the example on the lecture slides as well.

In [ ]:
k_minus_one_itemsets = {
    ('bread', 'cereal'), ('bread', 'milk'), ('bread', 'yogurt'), ('cereal', 'milk'),
    ('cereal', 'yogurt'), ('cheese', 'milk'), ('cheese', 'yogurt'), ('milk', 'yogurt')
}

one_itemsets = {('bread',), ('cereal',), ('cheese',), ('milk',), ('yogurt',)}

Lk = generate_Lk(k_minus_one_itemsets, one_itemsets)

for itemset in Lk:
    print(itemset)

#### 3.2 b) Generating Frequent Itemsets $F_{1}$, $F_{2}$, $F_{3}$, ... (5 Points)

Once we have a candidate set L of a given iteration, we can directly derive the frequent itemsets by removing all candidates with insufficient support. As this is a trivial filter step, we give you this method `generate_F` for free.

In [ ]:
def generate_Fk(Lk, transactions, minsup):
    return set([ s for s in Lk if support(transactions, s) >= minsup])

In [ ]:
Fk = generate_Fk(Lk, transactions_demo, 0.4)

for itemset in Fk:
    print(itemset)

**Note:** Just as a quick reminder, this filter step is actually the most costly part of the algorithms since we have to calculate the support for each candidate itemset which requires us to go through each transaction. For our toy dataset, this doesn't really matter, but the whole goal of the Apriori algorithm is to minimize the number of access to the datasets.

With the 2 methods `generate_L_kplus1` and `generate_F`, you now have everything to find all frequent itemsets using the Apriori algorithm. To this end, complete the method `find_frequent_itemsets_apriori` below!

In [ ]:
def find_frequent_itemsets_apriori(transactions, minsup):
    # The 1-itemsets are just all unique items across all transactions
    items = unique_items(transactions)
    
    ############################################################################################
    ### Your code starts here ##################################################################
    

    
    ### Your code ends here ####################################################################
    ############################################################################################
    
    # Initialize dictionary with all current frequent itemsets for each size k
    # Example format: { 1: {(a), (b), (c)}, 2: {(a, c), ...} }
    F = { 1: F_1 }
    
    for k in range(2, len(one_itemsets)+1):

        Fk = set()
        
        ########################################################################################
        ### Your code starts here ##############################################################


                
        ### Your code ends here ################################################################
        ########################################################################################
                
        F[k] = Fk    

    # Merge the dictionary of k-itemsets to a single set and return it
    # Example: {1: {(a), (b), (c)}, 2: (a, c)} => {(a), (b), (c), (a, c)}
    return set.union(*[ itemsets for k, itemsets in F.items() ])

Let's test your implementation using the code cell below. Of course, the result should match the brute force approach (assuming the same value for input parameter `minsup`; default: `0.4` to match the example in the lecture).

In [ ]:
F = find_frequent_itemsets_apriori(transactions_demo, 0.4)

for itemset in F:
    print(itemset)

#### From Frequent Itemsets to Association Rules

Your implementation so far gives you the Frequent Itemsets in a list of transactions using the Apriori method. This step is typically the most time-consuming one in Association Rule Mining. However, we still have to do the second step and find all Association Rules given the Frequent Itemsets. We saw in the lecture that this can also be done in an efficient manner using the Apriori method to avoid checking all rules.

Since this step is typically less computationally expensive, we simply do it the naive way -- that is, we go over all Frequent Itemsets, and check for Frequent Itemset and check which of the Association Rules that can be generated from it has a sufficiently high confidence. With all the auxiliary methods we provide, this becomes trivial to implement, so we simply give you the method `find_association_rules()` below. Note how it uses your implementation of `frequent_itemsets_apriori()`.

In [ ]:
def find_association_rules(transactions, min_support, min_confidence):
    # Initialize empty list of association rules
    association_rules = []
    
    # Find and loop over all frequent itemsets
    for itemset in find_frequent_itemsets_apriori(transactions, min_support):
        if len(itemset) == 1:
            continue

        # Find and loop over all association rules that can be generated from the itemset
        for r in generate_association_rules(itemset):
            # Check if the association rule fulfils the confidence requriement
            if confidence(transactions, r) >= min_confidence:
                association_rules.append(r)
                
    # Return final list of association rules
    return association_rules

find_association_rules(transactions_demo, 0.6, 1.0)

If everything is correct, for the default values for `min_support` and `min_confidence`, the one Association Rules that should be returned is $\{cereal\}\Rightarrow \{milk\}$ (in Python represented as a tuple of 2 tuples, left-hand side and right-hand side).

#### Comparison with `efficient-apriori` package

You can run the apriori algorithm over the demo data to check if your implementation is correct. Try different values for the parameters `min_support` and `min_confidence` and compare the results. Note that the order of the returned association rules might differ between your implementation and the apriori one.

In [ ]:
_, rules = apriori(transactions_demo, min_support=0.6, min_confidence=1.0, max_length=4)

for r in rules:
    print('Rule [{} => {}] (support: {}, confidence: {}, lift: {})'.format(r.lhs, r.rhs, r.support, r.confidence, r.lift))

### 3.3 Association Rule Mining over Real-World Datasets

For the last task, we look into real-world datasets to find interesting Association Rules. As those datasets are already quite large -- although not really large -- we only rely only on the `efficient-apriori` package for performance reasons. To make things even easier for you, we provide the method `show_top_rules` which computes the Association Rules using the `efficient-apriori` package, but (a) filters the rules w.r.t. to the right-hand side (optional), (b) sorts the rules w.r.t. the specified metric, and (c) shows only the top-k rules.

#### Loading the Data

For the first dataset, we consider a typical [Market Basket Analysis](https://www.kaggle.com/yugagrawal95/market-basket-analysis-apriori-in-python/data?select=Market_Basket_Optimisation.csv) dataset containing the set of items people have purchased in a supermarket. This dataset is very similar to the running example used throughout the lecture -- well, just larger :), with ~7,500 transactions.

In [ ]:
transactions_groceries = []

with open('data/a2-groceries.csv') as file:
    for line in file:
        line = line.strip()
        transactions_groceries.append(line.split(','))

print('Number of transactions overall: {}'.format(len(transactions_groceries)))
print()
print('Some example transactions:')
for t in transactions_groceries[:5]:
    print(t)

As the second dataset, use the [Coronavirus Disease 2019 (COVID-19) Clinical Data Repository](https://covidclinicaldata.org/) to find Association Rules that might tell as, which symptoms are most indicative of a COVID-19 infections. We already downloaded, cleaned, and prepared the dataset for you. The dataset file `data/a1-covid-symptoms-result.csv` contains over 710k transactions. Each transaction is a set of $0..n$ symptoms and $1$ test result label ("POSITIVE" and "NEGATIVE"). For example a line in the file can look like `runny_nose sore_throat fatigue POSITIVE`. Note that a line might also be just `NEGATIVE` or `POSITIVE` in case a person was tested without any symptoms. Feel free to take a look at the file -- looking at the raw data is always a good first step when it comes to data mining.

Note that we generate 2 lists of transactions which we both use later:
* `transactions_covid_all` contains all 710k+ transactions in the dataset
* `transactions_covid_pos` contains all 11k+ transactions with a "POSITIVE" test result label

In [ ]:
transactions_covid_all = []
transactions_covid_pos = []

vocab = set()

with open('data/a2-covid-symptoms-result.csv') as file:
    for line in file:
        line = line.strip()
        
        if 'POSITIVE' in line:
            transactions_covid_pos.append(line.split(' '))
        
        transactions_covid_all.append(line.split(' '))

print('Number of transactions overall: {}'.format(len(transactions_covid_all)))
print('Number of "POSITIVE" transactions: {}'.format(len(transactions_covid_pos)))
print()
print('Some example transactions:')
for t in transactions_covid_pos[:5]:
    print(t)

#### Finding Association Rules

Now that we have the data, we can try to find interesting Association Rules in both datasets. To this end, we have prepared a series of code cells to run the Apriori Algorithm implemented by the `efficient-apriori` package over both datasets with different parameters. Each run is labeled A-D; the question at the may make references to some of these individual runs.

**Note:** There is nothing for you to implement here, you can just run the code cells! However, for the questions below, you can always change the different parameters (e.g., `min_support`, `min_confidence`) to see their effects in the result as well as the runtime.

In [ ]:
%%time
# Run A
show_top_rules(transactions_covid_all, min_support=0.001, min_confidence=0.2, k=5, sort='lift', rhs='POSITIVE')

In [ ]:
%%time
# Run B
show_top_rules(transactions_covid_pos, min_support=0.15, min_confidence=0.8, k=5, sort='lift', rhs='POSITIVE')

In [ ]:
%%time
# Run C
show_top_rules(transactions_groceries, min_support=0.001, min_confidence=0.2, k=5, sort='lift')

In [ ]:
%%time
# Run D
show_top_rules(transactions_groceries, min_support=0.01, min_confidence=0.2, k=5, sort='lift')

### Questions

**Question 3.3 a) (3 Points)** From your observation, what are the effects of increasing/reducing `min_support` and `min_confidence`? Support your answer with evidence. You can perform more runs of efficient-apriori with different parameter settings, if needed.

**Your Answer:**

**Question 3.3 b) (3 Points)** Runs A and B return association rules with symptoms on the left-hand side and a POSITIVE test result on the right-hand side. As such both runs find rules which (combination of) symptoms are most indicative of a positive test result. However, the results of Run A and B are rather different. Explain the differences and discuss which result provides more reliable insights!

**Your Answer:**

**Question 3.3 c) (2 Points)** Runs A and C roughly return the same number of rules in roughly the similar amount of time. However, there are almost 10 times more transactions in the COVID dataset (Run A) compared to the groceries dataset (Run C). Explain why the result and performance are very similar despite the very different number of transactions.

**Important:** The exact runtimes might vary noticeable depending on your PC/laptop. So if the runtimes of both runs are not similar for you, please simply assume that the two runtimes are similar!

**Your Answer:**

**Question 3.3 d) (2 Points)** Have a look at the confidence and lift of the top rules for Run A and B. What noticeable differences can you observe and how can you explain these differences?

**Your Answer:**